# ETL Project: Suicides Rates And Mental Health All Over The Wolrd

In [573]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
!pip install mysqlclient

# Cleanup Suicides Rates Data

In [574]:
# Store CSV into DataFrame
#suicide data
csv_file = "./Resources/suicide_rates_overview_1985_to_2016.csv"
suicide_df = pd.read_csv(csv_file)
suicide_df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [575]:
#check columns names
suicide_df.columns

Index(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype='object')

In [576]:
# Extract 2014 to 2015 Data
three_years_suicides = suicide_df.loc[(suicide_df["year"]>=2014) & (suicide_df["year"]<2016),["country","year","sex","age",
                                   "suicides_no","population"]]
three_years_suicides.head()

,country,year,sex,age,suicides_no,population
564,Antigua and Barbuda,2014,female,15-24 years,0,8537
565,Antigua and Barbuda,2014,female,25-34 years,0,7578
566,Antigua and Barbuda,2014,female,35-54 years,0,15273
567,Antigua and Barbuda,2014,female,5-14 years,0,8296
568,Antigua and Barbuda,2014,female,55-74 years,0,6085


# Suicides data by countries

In [577]:
# Different countries numbers of suicidies and population
country_suicides = three_years_suicides.groupby(["country"])["suicides_no","population"].sum()
countries_suicides = pd.DataFrame(country_suicides).reset_index()

countries_suicides["percentage"] = countries_suicides["suicides_no"]/countries_suicides["population"] 
countries_suicides['id'] = countries_suicides.index
countries_suicides.head()

,country,suicides_no,population,percentage,id
0,Antigua and Barbuda,1,182726,0.000005,0
1,Argentina,6304,78976179,0.000080,1
2,Armenia,132,5598364,0.000024,2
3,Australia,5918,44168372,0.000134,3
4,Austria,2565,16360551,0.000157,4


# Suicides data by years

In [578]:
#Groupby year to caculate total number of suicides and population
groupby_year_suicides = three_years_suicides.groupby(["year"])["suicides_no","population"].sum()
years_suicides = pd.DataFrame(groupby_year_suicides).reset_index()
years_suicides["percentage"] = years_suicides["suicides_no"]/years_suicides["population"] 
years_suicides['id'] = years_suicides.index
years_suicides

,year,suicides_no,population,percentage,id
0,2014,222984,1912057309,0.000117,0
1,2015,203640,1774657932,0.000115,1


In [579]:
years_suicides.columns

Index(['year', 'suicides_no', 'population', 'percentage', 'id'], dtype='object')

# Suicides data by ages

In [580]:
#Groupby age to caculate total number of suicides and population
# sort age range
groupby_age_suicides = three_years_suicides.groupby(["age"])["suicides_no","population"].sum()
reset_index = pd.DataFrame(groupby_age_suicides).reset_index()

move_index_ages_suicides = pd.concat([reset_index.iloc[[3],:], reset.drop(3, axis=0)], axis=0)

move_index_ages_suicides["percentage"] = move_index_ages_suicides["suicides_no"]/move_index_ages_suicides["population"] 

move_index_ages_suicides

,age,suicides_no,population,percentage
3,5-14 years,3563,514876318,0.000007
0,15-24 years,47064,569779635,0.000083
1,25-34 years,69569,604626515,0.000115
2,35-54 years,156196,1119392551,0.000140
4,55-74 years,115999,760689229,0.000152
5,75+ years,49836,249452889,0.000200


In [581]:
move_index_ages_suicides.columns

Index(['age', 'suicides_no', 'population', 'percentage'], dtype='object')

In [582]:
#Move years from age column
move_index_ages_suicides[["age", "years"]] = move_index_ages_suicides.age.str.split(" ", expand = True)

move_index_ages_suicides_df = pd.DataFrame(move_index_ages_suicides).reset_index()
move_index_ages_suicides_df['id'] = move_index_ages_suicides_df.index
ages_suicides = move_index_ages_suicides_df[["age","suicides_no","population","percentage","id"]]
ages_suicides


,age,suicides_no,population,percentage,id
0,5-14,3563,514876318,0.000007,0
1,15-24,47064,569779635,0.000083,1
2,25-34,69569,604626515,0.000115,2
3,35-54,156196,1119392551,0.000140,3
4,55-74,115999,760689229,0.000152,4
5,75+,49836,249452889,0.000200,5


# Suicides data by gender

In [583]:
#Groupby gender to caculate total number of suicides and population
groupby_gender_suicides = three_years_suicides.groupby(["sex"])["suicides_no","population"].sum()
gender_suicides = pd.DataFrame(groupby_gender_suicides).reset_index()
gender_suicides["percentage"] = gender_suicides["suicides_no"]/gender_suicides["population"] 
gender_suicides['id'] = gender_suicides.index
gender_suicides

,sex,suicides_no,population,percentage,id
0,female,98804,1885356670,0.000052,0
1,male,327820,1801358571,0.000182,1


# Cleanup Mental Health Data

In [584]:
# Store CSV into DataFrame
csv_file = "./Resources/mental_health_survey.csv"
mental_health_df = pd.read_csv(csv_file)
mental_health_df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [585]:
mental_health_df.columns

Index(['Timestamp', 'Age', 'Gender', 'Country', 'state', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')

In [586]:
#Extract columns that I need to use
mental_health_data = mental_health_df[["Timestamp","Age","Gender","Country","self_employed",
                                       "remote_work","family_history","treatment","seek_help"
                                     ]]
mental_health_data.head()

,Timestamp,Age,Gender,Country,self_employed,remote_work,family_history,treatment,seek_help
0,2014-08-27 11:29:31,37,Female,United States,NaN,No,No,Yes,Yes
1,2014-08-27 11:29:37,44,M,United States,NaN,No,No,No,Don't know
2,2014-08-27 11:29:44,32,Male,Canada,NaN,No,No,No,No
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,No,Yes,Yes,No
4,2014-08-27 11:30:22,31,Male,United States,NaN,Yes,No,No,Don't know


In [587]:
# Split column(Timestamp) to get year
mental_health_data["Year"]= [d.split('-')[0] for d in mental_health_data.Timestamp]
mental_health_data

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Timestamp,Age,Gender,Country,self_employed,remote_work,family_history,treatment,seek_help,Year
0,2014-08-27 11:29:31,37,Female,United States,NaN,No,No,Yes,Yes,2014
1,2014-08-27 11:29:37,44,M,United States,NaN,No,No,No,Don't know,2014
2,2014-08-27 11:29:44,32,Male,Canada,NaN,No,No,No,No,2014
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,No,Yes,Yes,No,2014
4,2014-08-27 11:30:22,31,Male,United States,NaN,Yes,No,No,Don't know,2014
5,2014-08-27 11:31:22,33,Male,United States,NaN,No,Yes,No,Don't know,2014
6,2014-08-27 11:31:50,35,Female,United States,NaN,Yes,Yes,Yes,No,2014
7,2014-08-27 11:32:05,39,M,Canada,NaN,Yes,No,No,No,2014
8,2014-08-27 11:32:39,42,Female,United States,NaN,No,Yes,Yes,No,2014
9,2014-08-27 11:32:43,23,Male,Canada,NaN,No,No,No,Don't know,2014


In [588]:
# Create age range
bins = [5,14,24,34,54,74,np.inf]
group_labels = ["5-14","15-24","25-34","35-54","55-74","75+"]

mental_health_data["Age Range"] = pd.cut(mental_health_data["Age"],bins, labels = group_labels)
mental_health_data.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Timestamp,Age,Gender,Country,self_employed,remote_work,family_history,treatment,seek_help,Year,Age Range
0,2014-08-27 11:29:31,37,Female,United States,NaN,No,No,Yes,Yes,2014,35-54
1,2014-08-27 11:29:37,44,M,United States,NaN,No,No,No,Don't know,2014,35-54
2,2014-08-27 11:29:44,32,Male,Canada,NaN,No,No,No,No,2014,25-34
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,No,Yes,Yes,No,2014,25-34
4,2014-08-27 11:30:22,31,Male,United States,NaN,Yes,No,No,Don't know,2014,25-34


In [589]:
# Drop columns Timestamp and Age
reorganized_mental_health_data = mental_health_data[["Year","Age Range","Gender","Country",
                                                        "self_employed","remote_work","family_history",
                                                        "treatment","seek_help"]]
reorganized_mental_health_data.head()

,Year,Age Range,Gender,Country,self_employed,remote_work,family_history,treatment,seek_help
0,2014,35-54,Female,United States,NaN,No,No,Yes,Yes
1,2014,35-54,M,United States,NaN,No,No,No,Don't know
2,2014,25-34,Male,Canada,NaN,No,No,No,No
3,2014,25-34,Male,United Kingdom,NaN,No,Yes,Yes,No
4,2014,25-34,Male,United States,NaN,Yes,No,No,Don't know


In [590]:
# change Gender value to lowercase, replace m to male, f to female, drop off unsex rows
reorganized_mental_health_data["Gender"] = reorganized_mental_health_data["Gender"].str.lower()
reorganized_mental_health_data.loc[reorganized_mental_health_data["Gender"] == 'm','Gender'] ="male"
reorganized_mental_health_data.loc[reorganized_mental_health_data["Gender"] == 'f','Gender'] ="female"
reorganized_mental_health_data

drop_unsex_gender = reorganized_mental_health_data.loc[(reorganized_mental_health_data["Gender"] == "female") | 
                    (reorganized_mental_health_data["Gender"] == "male"), ["Year","Age Range","Gender","Country",
                                                                         "self_employed","remote_work","family_history",
                                                                         "treatment","seek_help"
                                                                    ]]
drop_unsex_gender.head()


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Year,Age Range,Gender,Country,self_employed,remote_work,family_history,treatment,seek_help
0,2014,35-54,female,United States,NaN,No,No,Yes,Yes
1,2014,35-54,male,United States,NaN,No,No,No,Don't know
2,2014,25-34,male,Canada,NaN,No,No,No,No
3,2014,25-34,male,United Kingdom,NaN,No,Yes,Yes,No
4,2014,25-34,male,United States,NaN,Yes,No,No,Don't know


In [591]:
# rename columns names
final_mental_data = drop_unsex_gender.rename(columns={"Year":"year","Age Range":"age","Gender":"sex","Country":"country"})
final_mental_data.head()

,year,age,sex,country,self_employed,remote_work,family_history,treatment,seek_help
0,2014,35-54,female,United States,NaN,No,No,Yes,Yes
1,2014,35-54,male,United States,NaN,No,No,No,Don't know
2,2014,25-34,male,Canada,NaN,No,No,No,No
3,2014,25-34,male,United Kingdom,NaN,No,Yes,Yes,No
4,2014,25-34,male,United States,NaN,Yes,No,No,Don't know


In [592]:
# drop NaN and Don't know data
cleanup_self_employed= final_mental_data.loc[(final_mental_data["self_employed"] == "No") 
                                             | (final_mental_data["self_employed"] == "Yes")
                                            ]

cleanup_mental_data = cleanup_self_employed.loc[(cleanup_self_employed["seek_help"] == "No") 
                                             | (cleanup_self_employed["seek_help"] == "Yes")
                                            ]
mental_data = pd.DataFrame(cleanup_mental_data)
mental_data.head()

,year,age,sex,country,self_employed,remote_work,family_history,treatment,seek_help
19,2014,35-54,male,France,Yes,Yes,Yes,No,No
20,2014,25-34,male,United States,No,No,Yes,Yes,No
21,2014,25-34,male,United States,Yes,Yes,No,No,No
22,2014,35-54,male,United States,No,Yes,No,Yes,No
24,2014,25-34,male,United States,No,No,Yes,Yes,Yes


In [593]:
# Rename columns values: Yes to 1, No to 0
mental_data.loc[mental_data["self_employed"] == 'No','self_employed'] = 0
mental_data.loc[mental_data["self_employed"] == 'Yes','self_employed'] = 1


mental_data.loc[mental_data["remote_work"] == 'No','remote_work'] = 0
mental_data.loc[mental_data["remote_work"] == 'Yes','remote_work'] = 1

mental_data.loc[mental_data["family_history"] == 'No','family_history'] = 0
mental_data.loc[mental_data["family_history"] == 'Yes','family_history'] = 1

mental_data.loc[mental_data["treatment"] == 'No','treatment'] = 0
mental_data.loc[mental_data["treatment"] == 'Yes','treatment'] = 1

mental_data.loc[mental_data["seek_help"] == 'No','seek_help'] = 0
mental_data.loc[mental_data["seek_help"] == 'Yes','seek_help'] = 1

number_mental_data = mental_data

number_mental_data.head()

,year,age,sex,country,self_employed,remote_work,family_history,treatment,seek_help
19,2014,35-54,male,France,1,1,1,0,0
20,2014,25-34,male,United States,0,0,1,1,0
21,2014,25-34,male,United States,1,1,0,0,0
22,2014,35-54,male,United States,0,1,0,1,0
24,2014,25-34,male,United States,0,0,1,1,1


# Total numbers of people that are involved in this mental health data

In [594]:
# Check how many people total are involved in this data
total_rows = len(number_mental_data)
print (f"Total numbers of people that are included in this mental health data are {total_rows}")

Total numbers of people that are included in this mental health data are 843


# Mental health data by countries

In [595]:
#Groupby country and get sum for self employed, remote work, family history, treatment and seek help
groupby_country_metal = number_mental_data.groupby(["country"]).sum()
countries_mental = pd.DataFrame(groupby_country_metal).reset_index()
countries_mental['id'] = countries_mental.index
countries_mental.head()

,country,self_employed,remote_work,family_history,treatment,seek_help,id
0,Australia,2,7,10,12,8,0
1,Austria,0,1,0,0,0,1
2,Belgium,0,0,1,1,0,2
3,Brazil,2,1,2,2,0,3
4,Bulgaria,0,2,0,2,0,4


In [596]:
countries_mental.columns

Index(['country', 'self_employed', 'remote_work', 'family_history',
       'treatment', 'seek_help', 'id'],
      dtype='object')

# Mental health data by years

In [597]:
#Groupby year and get sum for self employed, remote work, family history, treatment and seek help
groupby_year_metal = number_mental_data.groupby(["year"]).sum()
years_mental = pd.DataFrame(groupby_year_metal).reset_index()
years_mental['id'] = years_mental.index
years_mental

,year,self_employed,remote_work,family_history,treatment,seek_help,id
0,2014,108,231,303,401,220,0
1,2015,4,15,24,34,16,1
2,2016,0,0,1,1,0,2


# Mental health data by ages

In [598]:
#Groupby ages and get sum for self employed, remote work, family history, treatment and seek help
groupby_age_metal = number_mental_data.groupby(["age"]).sum()
ages_mental = pd.DataFrame(groupby_age_metal).reset_index()
ages_mental['id'] = ages_mental.index
ages_mental

,age,self_employed,remote_work,family_history,treatment,seek_help,id
0,5-14,1,1,0,0,0,0
1,15-24,8,16,39,46,19,1
2,25-34,59,126,176,230,111,2
3,35-54,42,97,110,150,96,3
4,55-74,2,4,3,8,9,4
5,75+,0,1,0,1,0,5


# Mental health data by gender

In [599]:
#Groupby sex and get sum for self employed, remote work, family history, treatment and seek help
groupby_gender_metal = number_mental_data.groupby(["sex"]).sum()
gender_mental = pd.DataFrame(groupby_gender_metal).reset_index()
gender_mental['id'] = gender_mental.index
gender_mental

,sex,self_employed,remote_work,family_history,treatment,seek_help,id
0,female,13,39,87,110,52,0
1,male,99,207,241,326,184,1


In [600]:
gender_mental.columns

Index(['sex', 'self_employed', 'remote_work', 'family_history', 'treatment',
       'seek_help', 'id'],
      dtype='object')

# Connect to workbench

In [601]:
# create engine to connect to workbench
connection_string = "root:0502@127.0.0.1/suicide_mental_health_db"
engine = create_engine(f'mysql://{connection_string}')

In [602]:
# Check for tables
engine.table_names()

['ages_mental',
 'ages_suicides',
 'countries_mental',
 'countries_suicides',
 'gender_mental',
 'gender_suicides',
 'years_mental',
 'years_suicides']

In [604]:
# Use pandas to load csv converted DataFrame into database
countries_suicides.to_sql(name='countries_suicides', con=engine, if_exists='append', index=False)

In [494]:
# Use pandas to load csv converted DataFrame into database
years_suicides.to_sql(name='years_suicides', con=engine, if_exists='append', index=False)

In [571]:
# Use pandas to load csv converted DataFrame into database
ages_suicides.to_sql(name='ages_suicides', con=engine, if_exists='append', index=False)

In [497]:
# Use pandas to load csv converted DataFrame into database
gender_suicides.to_sql(name='gender_suicides', con=engine, if_exists='append', index=False)

In [499]:
# Use pandas to load csv converted DataFrame into database
countries_mental.to_sql(name='countries_mental', con=engine, if_exists='append', index=False)

In [500]:
# Use pandas to load csv converted DataFrame into database
years_mental.to_sql(name='years_mental', con=engine, if_exists='append', index=False)

In [501]:
# Use pandas to load csv converted DataFrame into database
ages_mental.to_sql(name='ages_mental', con=engine, if_exists='append', index=False)

In [506]:
# Use pandas to load csv converted DataFrame into database
gender_mental.to_sql(name='gender_mental', con=engine, if_exists='append', index=False)

# Confirm data has been added by querying the table

In [507]:
# Confirm data has been added by querying the table
# NOTE: can also check using pgAdmin
pd.read_sql_query('select * from countries_suicides', con=engine).head()

,id,country,suicides_no,population,percentage
0,0,Antigua and Barbuda,1,182726,0.000005472674934054267
1,1,Argentina,6304,78976179,0.00007982153707385615
2,2,Armenia,132,5598364,0.000023578316808267557
3,3,Australia,5918,44168372,0.00013398727940436655
4,4,Austria,2565,16360551,0.00015677956078618624


In [508]:
# Confirm data has been added by querying the table
pd.read_sql_query('select * from years_suicides', con=engine).head()

,id,year,suicides_no,population,percentage
0,0,2014,222984,1912057309,0.00011661993547495705
1,1,2015,203640,1774657932,0.00011474887431996669


In [572]:
# Confirm data has been added by querying the table
pd.read_sql_query('select * from ages_suicides', con=engine).head()

,id,age,suicides_no,population,percentage
0,0,5-14,3563,514876318,0.0000069201085298314305
1,1,15-24,47064,569779635,0.00008260035478452999
2,2,25-34,69569,604626515,0.0001150611133883204
3,3,35-54,156196,1119392551,0.00013953639396694538
4,4,55-74,115999,760689229,0.00015249197119892438


In [510]:
# Confirm data has been added by querying the table
pd.read_sql_query('select * from gender_suicides', con=engine).head()

,id,sex,suicides_no,population,percentage
0,0,female,98804,1885356670,0.00005240599912588423
1,1,male,327820,1801358571,0.0001819848670206816


In [511]:
# Confirm data has been added by querying the table
pd.read_sql_query('select * from countries_mental', con=engine).head()

,id,country,self_employed,remote_work,family_history,treatment,seek_help
0,0,Australia,2,7,10,12,8
1,1,Austria,0,1,0,0,0
2,2,Belgium,0,0,1,1,0
3,3,Brazil,2,1,2,2,0
4,4,Bulgaria,0,2,0,2,0


In [512]:
# Confirm data has been added by querying the table
pd.read_sql_query('select * from years_mental', con=engine).head()

,id,year,self_employed,remote_work,family_history,treatment,seek_help
0,0,2014,108,231,303,401,220
1,1,2015,4,15,24,34,16
2,2,2016,0,0,1,1,0


In [513]:
# Confirm data has been added by querying the table
pd.read_sql_query('select * from ages_mental', con=engine).head()

,id,age,self_employed,remote_work,family_history,treatment,seek_help
0,0,5-14,1,1,0,0,0
1,1,15-24,8,16,39,46,19
2,2,25-34,59,126,176,230,111
3,3,35-54,42,97,110,150,96
4,4,55-74,2,4,3,8,9


In [514]:
# Confirm data has been added by querying the table
pd.read_sql_query('select * from gender_mental', con=engine).head()

,id,sex,self_employed,remote_work,family_history,treatment,seek_help
0,0,female,13,39,87,110,52
1,1,male,99,207,241,326,184
